selenium, bs4를 설치해 브라우저를 원격 조정하자.

컴퓨터가 최소 8GB 이상, i5 이상의 CPU, Windows 환경을 추천한다.

Update: 2020/08/11 - remove previous comments 부분 해결하기. click more 부분 처리(bug fix 필요)

In [ ]:
!pip install selenium

In [ ]:
!pip install bs4

Firefox를 설치하자.

다음으로 geckodriver를 설치해야 한다.

https://github.com/mozilla/geckodriver/releases 에서 본인에게 맞는 OS를 골라 설치하자.

압축을 풀고 C:/Windows 폴더에 geckodriver.exe를 복사한다. 이때 관리자 권한이 필요하다. 리눅스나 mac사용자는 /usr/bin 으로 geckodriver를 옮기면 된다.

예를 들어, Linux 사용자는 터미널에서
```
    $ wget https://github.com/mozilla/geckodriver/releases/download/v0.27.0/geckodriver-v0.27.0-linux64.tar.gz
    $ sudo sh -c 'tar -x geckodriver -zf geckodriver-v0.27.0-linux64.tar.gz -O > /usr/bin/geckodriver'
    $ sudo chmod +x /usr/bin/geckodriver
    $ rm geckodriver-v0.73.0-linux64.tar.gz
```



In [42]:
from selenium import webdriver #웹드라이브
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.firefox.options import Options #Firefox options
from selenium.webdriver.common.keys import Keys #key emulation
from bs4 import BeautifulSoup as BS

In [2]:
import time
import random
import hashlib

In [3]:
'''
Factory - Firefox
'''
def fire_fox_browser(headless=False,image_allow=True):
    options=Options()
    profile=webdriver.FirefoxProfile()
    profile.set_preference('permissions.default.image',1 if image_allow else 2)
    options.headless=headless
    driver=webdriver.Firefox(firefox_profile=profile,options=options)
    return driver

In [304]:
youtube_url="https://www.youtube.com/watch?v=9kU6jb8XXpw&autoplay=1" #펭TV 남극 동물의 한국식 여름 휴가, 코멘트 많은
youtube_url_no_comment="https://www.youtube.com/watch?v=S2qJAO03SsU" #코멘트 없는
youtube_url_few_comment="https://www.youtube.com/watch?v=-5WjyN53Zw4" #코멘트 적은

In [4]:
browser=fire_fox_browser(image_allow=True)

In [8]:
browser.quit()

In [118]:
class Comment:
    AUTHOR='author-text'
    CONTENT='content-text'
    SREPLIES='following-sibling::*[1]'
    REPLIES='ytd-comment-renderer'
    def __init__(self,obj,r):
        self.r=r
        self.id=''
        self.obj=obj
        self.author=''
        self.content=''
        self.comments_of_comment=[]
        self.get_uid()
        self.process_author(r)
        self.process_content(r)
        self.process_replies(r)
        self.parent_id=obj.id
    def get_uid(self):
        selfid=str(id(self))+"/"
        rint=str(random.randint(0,100))+'/'
        idcode=selfid+rint
        self.id=hashlib.md5(idcode.encode()).hexdigest()
    def process_author(self,r):
        try:
            self.author=r.find_element_by_id(Comment.AUTHOR).text
        except NoSuchElementException:
            pass
    def process_content(self,r):
        try:
            self.content=r.find_element_by_id(Comment.CONTENT).text
        except NoSuchElementException:
            pass
    def process_replies(self,r):
        try:
            replies=r.find_element_by_xpath(Comment.SREPLIES)
            rs=replies.find_elements_by_tag_name(Comment.REPLIES)
            if len(rs) > 0: #if any
                for rss in rs:
                    self.comments_of_comment.append(Comment(self,rss))
        except NoSuchElementException:
            pass
class Comments:
    def __init__(self):
        self.id=''
        self.parent=None
        self.comments=[]
    def analyze_data(self,browser):
        pass
    def get_uid(self,browser):
        pass
class YouTubeComments(Comments):
    COMMENT='comment'
    def __init__(self):
        super().__init__()
    def get_uid(self,browser):
        url=browser.current_url+"/"
        selfid=str(id(self))+"/"
        rint=str(random.randint(0,100))+'/'
        idcode=url+selfid+rint
        self.id=hashlib.md5(idcode.encode()).hexdigest()
    def analyze_data(self,browser):
        self.get_uid(browser) #generate id
        replies=browser.find_elements_by_id(YouTubeComments.COMMENT)
        if len(replies) <=0: #nothing found
            return False
        #processing...
        for r in replies:
            comment=Comment(self,r)
            if comment.author is not '':
                self.comments.append(comment)      

In [313]:
# FactoryFireFox.py
# Author: Taekyung Kim(kimtk@office.kw.ac.kr)
# First created on 6 Aug 2020
# Last updated on 11 Aug 2020   
# YouTube Data Collection
# Dependencies:
#  pip install selenium
#  pip install bs4
# Install Firefox first.

from selenium import webdriver #webdriver
from selenium.webdriver.firefox.options import Options #Firefox options
from selenium.webdriver.common.keys import Keys #key emulation
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException,TimeoutException #exception
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait

from bs4 import BeautifulSoup as BS
# utility
import time
import random
import hashlib
import sys
#
from YouTubeComment import YouTubeComments,Comment
#
#' Firefox factor, inherited from webdriver.Firefox
class FactoryFirefoxDriver(webdriver.Firefox):
    def __init__(self,headless=False,image_allow=True):
        options=Options()
        profile=webdriver.FirefoxProfile()
        profile.set_preference('permissions.default.image',1 if image_allow else 2)
        options.headless=headless #headless mode on?
        super().__init__(firefox_profile=profile,options=options)
#' Webdriver for YouTube comment collection
#' >>> b=YouTubeCommentBrowser(headless=True,image_allow=False)
#' >>> b.visit("some YouTube video URL")
#' >>> comments=b.collect_comments(verbose=True)
#' >>> b.quit() #session close.
#' 
def check_scroll_presence(browser,tb=5):
    try:
        WebDriverWait(browser,tb).until(EC.presence_of_element_located((By.TAG_NAME,'html')))
    except TimeoutException:
        print("Scrollbar exception")
        return False
    return True
class YouTubeCommentBrowser(FactoryFirefoxDriver):
    def __init__(self,headless=False,image_allow=True):
        self.comments=[] #Comment object
        super().__init__(headless,image_allow)
    def stop_player(self,tb=1):
        time.sleep(tb)
        self.find_element_by_tag_name('html').send_keys('k') #stop video
    def press_up(self,h,tb=1):
        h.send_keys(Keys.HOME)
        time.sleep(tb)
    def press_down(self,h,tb=1):
        h.send_keys(Keys.END)
        time.sleep(tb)
    def scroll_up(self,tb=1):
        try:
            WebDriverWait(self,tb).until(EC.presence_of_element_located((By.TAG_NAME,'html')))
        except TimeoutException:
            print("Scrollbar exception")
            return False
        html=self.find_element_by_tag_name('html')
        self.press_up(html,tb)
    def scroll_down(self,tb=5):
        initial_height = self.execute_script("return window.scrollY")
        html=self.find_element_by_tag_name('html')
        self.press_down(html,tb)
        time.sleep(tb)
        last_height = self.execute_script("return window.scrollY")
        if initial_height == last_height:
            return False
        while True:
            time.sleep(tb)
            new_height=self.execute_script("return window.scrollY")
            if new_height==last_height:
                break
            last_height=new_height
        return True
    def remove_element_by_id(self,s,tb=0.5):
        script="document.getElementById('{}').remove();".format(s)
        self.execute_script(script)
        time.sleep(tb)
    def remove_comments(self,tb=0.5):
        script='''
        document.querySelectorAll("ytd-comment-thread-renderer").forEach(function(item,index){item.innerHTML='<span id="comment"></span>';});
        '''
        self.execute_script(script)
        time.sleep(tb)
    def reply_more(self,tb=1):
        code_view='//yt-formatted-string[starts-with(.,"View") and contains(.,"reply")]'
        code_show_more_replies='//yt-formatted-string[starts-with(.,"View") and contains(.,"replies")]'
        eles=self.find_elements_by_xpath(code_view)
        for ele in eles:
            try:
                if ele.text is not "":
                    ele.click()
                    time.sleep(0.1)
            except:
                pass
        eles=self.find_elements_by_xpath(code_show_more_replies)
        for ele in eles:
            try:
                if ele.text is not "":
                    ele.click()
                    time.sleep(0.1)
            except:
                pass
        time.sleep(tb)
    def text_more(self,tb=1):
        code_read_more='//span[starts-with(.,"Read more")]'
        eles=self.find_elements_by_xpath(code_read_more)
        for ele in eles:
            try:
                if ele.text is not "":
                    ele.click()
                    time.sleep(0.1)
            except:
                pass
        time.sleep(tb)
    def visit(self,url,remove_ad=True,remove_player=False,tb=10):
        self.get(url)
        self.stop_player(0.5)
        self.stop_player(0.5)
        if remove_ad:
            self.remove_element_by_id('related')
            time.sleep(0.1)
        if remove_player:
            self.remove_element_by_id('player-container-outer')
            time.sleep(0.1)
    def get_data(self):
        page=self.page_source
        soup=BS(page,'html.parser')
        return [ele.text for ele in soup.select('#content-text')]
    def collect_comments(self,verbose=False,remove_previous=True):
        if verbose:
            print("Comments are collecting now..")
        comments=YouTubeComments()
        comments.analyze_data(self)
        previous=len(self.comments)
        self.comments+=comments.comments #update
        after=len(self.comments)
        if previous==after:
            if verbose:
                print("..............................complete. Nothing new.")
            return False
        if verbose:
            print("..............................complete. Total={}".format(len(self.comments)))
        return True

In [315]:
b=YouTubeCommentBrowser(headless=False,image_allow=False)

In [316]:
b.visit(youtube_url,remove_player=False,tb=30)

In [318]:
loopnext=True
while loopnext:
    if check_scroll_presence(b) is False:
        print("CEHCK!")
        break
    b.scroll_down()
    b.reply_more()
    b.text_more()
    loopnext=b.collect_comments(verbose=True)
    b.remove_comments()
    time.sleep(1) #don't know why, but...
    b.scroll_up()

Comments are collecting now..
..............................complete. Total=480
Comments are collecting now..
..............................complete. Total=520
Comments are collecting now..
..............................complete. Total=560
Comments are collecting now..
..............................complete. Total=600
Comments are collecting now..
..............................complete. Total=640
Comments are collecting now..
..............................complete. Total=680
Comments are collecting now..
..............................complete. Total=720
Comments are collecting now..
..............................complete. Total=760
Comments are collecting now..
..............................complete. Total=800
Comments are collecting now..
..............................complete. Total=840
Comments are collecting now..
..............................complete. Total=880
Comments are collecting now..
..............................complete. Total=920
Comments are collecting now..
..........

In [319]:
cnt=0
for i in b.comments:
    cnt+=1
    cnt+=len(i.comments_of_comment)

In [320]:
cnt

1677

In [321]:
b.quit()